# RAG 시스템 보기


In [ ]:
import warnings
from retrieval import load_vectorstore_retriever_embeddings, rag
from model import llm_load

warnings.filterwarnings("ignore")

# 1. 벡터스토어 로드
vectorstore, retriever, embeddings = load_vectorstore_retriever_embeddings("RAG/vectorDB")

# 2. LLM 한 번만 로드 (재사용)
llm = llm_load()

# 3. RAG 실행
rag_output = rag(vectorstore, "What is the capital of France?", llm)

/home/humane/miniconda3/envs/knowledge_conflict/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


벡터스토어 로드 중
검색기 로드 중
검색기 로드 완료


Loading checkpoint shards: 100%|██████████| 5/5 [00:26<00:00,  5.31s/it]


RAG 실행 시작
vectorstore : <langchain_community.vectorstores.faiss.FAISS object at 0x7cc9259c5e80>
LLM 응답 생성 시작


OutOfMemoryError: CUDA out of memory. Tried to allocate 350.00 MiB. GPU 0 has a total capacity of 10.56 GiB of which 245.00 MiB is free. Including non-PyTorch memory, this process has 9.84 GiB memory in use. Of the allocated memory 9.20 GiB is allocated by PyTorch, and 460.75 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
rag_output